In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import matplotlib as plt

import numpy as np

In [13]:
# Definición del modelo

class VGG16(nn.Module):
  def __init__(self, num_classes = 1000):
    super(VGG16, self).__init__()
    self.characteristic = nn.Sequential(
        # Bloque 1
        nn.Conv2d(3, 64, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(64, 64, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Bloque 2
        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(128, 128, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Bloque 3
        nn.Conv2d(128, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(256, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(256, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Bloque 4
        nn.Conv2d(256, 512, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(512, 512, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(512, 512, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Bloque 5
        nn.Conv2d(512, 512, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(512, 512, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(512, 512, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )

    # Bloque 6
    self.flatter = nn.Sequential(
        nn.AdaptiveAvgPool2d((7, 7)),
        nn.Flatten()
    )

    self.classificator_nn = nn.Sequential(
        nn.Dropout(),
        nn.Linear(7 * 7 * 512, 4096),                       # FC - 1
        nn.ReLU(inplace = True),
        nn.Linear(4096, 4096),                              # FC - 2
        nn.ReLU(inplace = True),
        nn.Linear(4096, num_classes),                       # FC - 3
        nn.ReLU(inplace = True),
        nn.Softmax(dim = 1)                                 # Output
    )

  def forward(self, x_data):
    value_tracker = self.characteristic(x_data)
    value_tracker = self.flatter(value_tracker)
    value_tracker = value_tracker.view(value_tracker.size(0), -1)
    value_tracker = self.classificator_nn(value_tracker)
    return value_tracker

In [14]:
vgg_16 = VGG16()

In [15]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
])

In [16]:
'''
Se trae el dataset
'''

# Descargar y cargar el conjunto de datos CIFAR-10

# ~5200 datos
trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

# ~900 datos
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [17]:
'''
Hiperparametros
'''

iterations = 1
learning_rate = 0.01
momentum_value = 0.8
lost_criteria = nn.CrossEntropyLoss()
optimizator = optim.SGD(vgg_16.parameters(), lr = learning_rate, momentum = momentum_value)

In [18]:
"""
Analytics
"""

error = []

In [ ]:
'''
Trainig process
'''

for epochs in range(iterations):
  iteration_lost = 0
  for j, i in enumerate(trainloader, 0):
    X, Y = i
    # Se inicializan los gradientes
    optimizator.zero_grad()
    # Se pasa la data por toda la arquitectura alexnet (Fowarding data)
    output = vgg_16(X)
    # Se calcula la perdida del modelo
    lost = lost_criteria(output, Y)
    # Backward propagation
    lost.backward()
    # Se actualizan los pesos, es decir; se da un paso
    optimizator.step()
    # Para almacenar la perdida en cada epoca
    iteration_lost += lost

    if (j % 200 == 0):
      print('[{} {:5d} {:.3f}]'.format(epochs, j+1, iteration_lost / 200))
      error.append(iteration_lost)
      iteration_lost = 0

[0     1 0.035]
